# Podstawy Przetwarzania Dużych Zbiorów Danych

Grupa IBD4 Poniedziałki 8:30

Łukasz Moskwa 247739 

Artur Szewczykowski 252546



# Część 1
 
1. https://github.com/GoogleCloudPlatform/covid-19-open-data/blob/main/docs/table-by-sex.md
2. https://medium.com/artificial-intelligence-101/data-analysis-with-google-bigquery-exploring-the-covid-19-open-data-public-dataset-2d246c66422d

# Część 2

In [1]:
# 2.5 Uruchom klienta BigQuery z poziomu Pythona.

import os
from operator import index

import pandas as pd
from google.cloud import bigquery
 

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=".env/propane-flag-452609-u3-1feb9eb63dc4.json" # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client() 


In [3]:
# 2.6 Pobierz dane do obiektu DataFrame.

query = ('select * from bigquery-public-data.covid19_open_data.covid19_open_data limit 10')
query_job = client.query(query)    
query_result = query_job.result()  
df = query_result.to_dataframe()
df


,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,AR,2020-08-07,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
1,AR,2020-08-06,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
2,AR,2020-07-24,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
3,AR,2020-05-03,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
4,AR,2021-04-02,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
5,AR,2020-06-02,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
6,AR,2021-06-20,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
7,AR,2022-04-27,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
8,BR,2021-06-24,ChIJzyjM68dZnAARYz4p8gYVWik,Q155,country/BRA,BR,Brazil,BR,BRA,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-53 -14)
9,BR,2021-07-28,ChIJzyjM68dZnAARYz4p8gYVWik,Q155,country/BRA,BR,Brazil,BR,BRA,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-53 -14)


# Część 3
Misja dodatkowa - Poznanie danych

In [4]:
# 3.1  Sprawdź, ile jest zapisanych wierszy z danymi. 

dataset_ref = client.dataset('covid19_open_data', project='bigquery-public-data')
table_ref = dataset_ref.table('covid19_open_data')
table = client.get_table(table_ref)

rows = table.num_rows

rows

22756333

In [5]:
# 3.2 Sprawdź, ile krajów jest uwzględnionych w danych. 

query = ("""
SELECT COUNT(DISTINCT country_code) AS num_countries
FROM bigquery-public-data.covid19_open_data.covid19_open_data

""")

query_job = client.query(query)
result = query_job.result()

number_of_country = result.to_dataframe()

# Zwraca 246, ponieważ niektóre tereny podległe takie jak "French Southern Territories", "Guam" itd.
# Co jest niby okey ale nie są to państwa bo ich jest 195
number_of_country.iloc[0, 0]


246

In [6]:
 # 3.3. Sprawdź, w jaki sposób zapisywane są dzienne informacje dla krajów.
 
# Dodają najczęsciej raz dziennie ale zdarza się, że kraj ma 2 wpisy ponieważ robi to dla innego aggregation_level
query = ("""
SELECT *
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE  aggregation_level = 0 AND country_name = 'Poland'
ORDER BY date ASC 
""")

country_all = client.query(query).to_dataframe()
country_all['date']

0      2020-01-01
1      2020-01-02
2      2020-01-03
3      2020-01-04
4      2020-01-05
          ...    
986    2022-09-13
987    2022-09-14
988    2022-09-15
989    2022-09-16
990    2022-09-17
Name: date, Length: 991, dtype: dbdate

In [7]:

is_continuous = (country_all['date'].diff().dropna() == pd.Timedelta(days=1)).all()

is_continuous


True

### 3.4. Sprawdź, w jaki sposób zapisywane są wartości liczbowe.

In [8]:
display(country_all.dtypes.value_counts())

float64    441
Int64      217
object      42
dbdate       1
Name: count, dtype: int64

Przykłady kolumn, które powiny być typem liczbowym a są obiektami czyli ciągami znaków.

In [9]:
# Wszystkie age_bins{1-9}
display(country_all[['population_largest_city', 'population_clustered', 'human_capital_index', 'area_rural_sq_km', 'area_urban_sq_km', 'life_expectancy']])

,population_largest_city,population_clustered,human_capital_index,area_rural_sq_km,area_urban_sq_km,life_expectancy
0,1775933,1775933,0.747,277840,30501,77.7
1,1775933,1775933,0.747,277840,30501,77.7
2,1775933,1775933,0.747,277840,30501,77.7
3,1775933,1775933,0.747,277840,30501,77.7
4,1775933,1775933,0.747,277840,30501,77.7
...,...,...,...,...,...,...
986,1775933,1775933,0.747,277840,30501,77.7
987,1775933,1775933,0.747,277840,30501,77.7
988,1775933,1775933,0.747,277840,30501,77.7
989,1775933,1775933,0.747,277840,30501,77.7


In [10]:
# Zapisane w typie coś / 1000
display(country_all.filter(like='_per_'))

,gdp_per_capita_usd,nurses_per_1000,physicians_per_1000,hospital_beds_per_1000
0,15222,6.8926,2.3788,NaN
1,15222,6.8926,2.3788,NaN
2,15222,6.8926,2.3788,NaN
3,15222,6.8926,2.3788,NaN
4,15222,6.8926,2.3788,NaN
...,...,...,...,...
986,15222,6.8926,2.3788,NaN
987,15222,6.8926,2.3788,NaN
988,15222,6.8926,2.3788,NaN
989,15222,6.8926,2.3788,NaN


###  3.5. Sprawdź, jaki przedział czasowy jest uwzględniony w danych. Dodatkowo porównaj przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób w danych.

In [11]:


query_new_confirmed = ("""
SELECT date, SUM(new_confirmed)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
GROUP BY date
HAVING SUM(new_confirmed) IS NOT NULL AND SUM(new_confirmed) > 0
ORDER BY date ASC
""")

query_new_deceased = ("""
SELECT date, SUM(new_deceased)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
GROUP BY date
HAVING SUM(new_deceased) IS NOT NULL AND SUM(new_deceased) > 0
ORDER BY date ASC
""")

query_new_person_vaccinated = ("""
SELECT date, SUM(new_persons_vaccinated)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
GROUP BY date
HAVING SUM(new_persons_vaccinated) IS NOT NULL AND SUM(new_persons_vaccinated) > 0
ORDER BY date ASC
""")

new_confirmed = client.query(query_new_confirmed).to_dataframe()
new_deceased = client.query(query_new_deceased).to_dataframe()
new_person_vaccinated = client.query(query_new_person_vaccinated).to_dataframe()

In [12]:
print(f"Przedział czasowy danych jest od {country_all['date'].min()} do {country_all['date'].max()}.")
print(f"Przedział czasowy new_confirmed jest od {new_confirmed['date'].min()} do {new_confirmed['date'].max()}.")
print(f"Przedział czasowy new_deceased jest od {new_deceased['date'].min()} do {new_deceased['date'].max()}.")
print(f"Przedział czasowy new_person_vaccinated jest od {new_person_vaccinated['date'].min()} do {new_person_vaccinated['date'].max()}.")


Przedział czasowy danych jest od 2020-01-01 do 2022-09-17.
Przedział czasowy new_confirmed jest od 2020-01-01 do 2022-09-15.
Przedział czasowy new_deceased jest od 2020-01-02 do 2022-09-15.
Przedział czasowy new_person_vaccinated jest od 2020-12-07 do 2022-09-14.


In [38]:
# Dla krajów z osobna
query = """
SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_confirmed' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_confirmed IS NOT NULL AND new_confirmed > 0
GROUP BY country_name

UNION ALL

SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_deceased' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_deceased IS NOT NULL AND new_deceased > 0
GROUP BY country_name

UNION ALL

SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_persons_vaccinated' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_persons_vaccinated IS NOT NULL AND new_persons_vaccinated > 0
GROUP BY country_name
ORDER BY country_name, metric
"""

dates_info = client.query(query).to_dataframe()
dates_info

,country_name,start_date,end_date,metric
0,Afghanistan,2020-02-22,2022-09-15,new_confirmed
1,Afghanistan,2020-03-21,2022-09-15,new_deceased
2,Afghanistan,2021-02-28,2022-09-12,new_persons_vaccinated
3,Albania,2020-03-07,2022-09-13,new_confirmed
4,Albania,2020-03-09,2022-09-11,new_deceased
...,...,...,...,...
663,Zambia,2020-01-02,2022-09-10,new_deceased
664,Zambia,2021-04-15,2022-08-07,new_persons_vaccinated
665,Zimbabwe,2020-01-02,2022-09-12,new_confirmed
666,Zimbabwe,2020-01-02,2022-08-29,new_deceased


###  3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń. 


1. Ile kolumn liczy zbiór danych

In [13]:
num_columns = country_all.shape[1]
print(f"Liczba kolumn: {num_columns}")

Liczba kolumn: 701


2. Które państwa podawały dane z podziałem na płeć

In [14]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_confirmed_male) AS confirmed_male, SUM(new_confirmed_female) AS confirmed_female, SUM(new_deceased_male) AS deceased_male, SUM(new_deceased_female) as deceased_female, SUM(new_tested_male) AS tested_male, SUM(new_tested_female) AS tested_female, SUM(new_hospitalized_patients_male) AS hospitalized_male, SUM(new_hospitalized_patients_female) AS hospitalized_female, SUM(new_intensive_care_patients_male) AS cared_male, SUM(new_intensive_care_patients_female) AS cared_female, SUM(new_recovered_male) AS recovered_male, SUM(new_recovered_female) AS recovered_female
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_confirmed_male) IS NOT NULL OR SUM(new_deceased_male) IS NOT NULL OR SUM(new_tested_male) IS NOT NULL OR SUM(new_hospitalized_patients_male) IS NOT NULL OR SUM(new_recovered_male) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

,Country_name,3_letter_code,confirmed_male,confirmed_female,deceased_male,deceased_female,tested_male,tested_female,hospitalized_male,hospitalized_female,cared_male,cared_female,recovered_male,recovered_female
0,Argentina,ARG,4418859,4780139,72801,53930,13178569,14928395,248464,222232,48422,33209,<NA>,<NA>
1,Brazil,BRA,8875904,8368673,311308,244105,26382353,26921275,996777,792931,330847,245941,0,0
2,Colombia,COL,2697358,3042821,93611,62159,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2542476,2898545
3,Czech Republic,CZE,1956993,2111418,23149,17783,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1925067,2082491
4,Estonia,EST,272596,326685,<NA>,<NA>,1595674,1890648,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,Finland,FIN,526714,587525,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,Germany,DEU,6092181,6356325,64006,56007,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4370388,4550358
7,Hong Kong,HKG,7417,<NA>,127,<NA>,<NA>,<NA>,6708,<NA>,<NA>,<NA>,<NA>,<NA>
8,India,IND,92449,51846,30,13,<NA>,<NA>,3431,1712,<NA>,<NA>,<NA>,<NA>
9,Malaysia,MYS,1676866,1323134,20855,15444,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


3. Które państwa podawały dane z podziałem na kategorie wiekowe

In [15]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_confirmed_age_0) AS confirmed_age_0, SUM(new_deceased_age_0) AS deceased_age_0, SUM(new_tested_age_0) AS tested_age_0, SUM(new_hospitalized_patients_age_0) as hospitalized_age_0, SUM(new_intensive_care_patients_age_0) AS cared_age_0, SUM(new_recovered_age_0) AS recovered_age_0 
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_confirmed_age_0) IS NOT NULL OR SUM(new_deceased_age_0) IS NOT NULL OR SUM(new_tested_age_0) IS NOT NULL OR SUM(new_hospitalized_patients_age_0) IS NOT NULL OR SUM(new_recovered_age_0) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

,Country_name,3_letter_code,confirmed_age_0,deceased_age_0,tested_age_0,hospitalized_age_0,cared_age_0,recovered_age_0
0,Argentina,ARG,224240,192,1303477,41095,3269,<NA>
1,Brazil,BRA,891485,1214,2726462,21195,5273,0
2,Colombia,COL,198102,360,<NA>,<NA>,<NA>,191387
3,Czech Republic,CZE,321365,7,<NA>,<NA>,<NA>,320168
4,Estonia,EST,44622,<NA>,208798,<NA>,<NA>,<NA>
5,Finland,FIN,85684,<NA>,<NA>,<NA>,<NA>,<NA>
6,France,FRA,2789829,<NA>,23039639,<NA>,<NA>,<NA>
7,Germany,DEU,437029,23,<NA>,<NA>,<NA>,290555
8,Hong Kong,HKG,666,0,<NA>,578,<NA>,<NA>
9,India,IND,4273,1,<NA>,78,<NA>,<NA>


4. Które państwa podawały dane dotyczące szczepień z podziałem na rodzaj(firmę) szczepionki

In [16]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_persons_fully_vaccinated_moderna) AS vaccinated_moderna, SUM(new_persons_fully_vaccinated_janssen) AS vaccinated_janssen, SUM(new_persons_fully_vaccinated_pfizer) AS vaccinated_pfizer, SUM(new_persons_vaccinated) AS total_vaccinated 
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_persons_fully_vaccinated_moderna) IS NOT NULL OR SUM(new_persons_fully_vaccinated_janssen) IS NOT NULL OR SUM(new_persons_fully_vaccinated_pfizer) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

,Country_name,3_letter_code,vaccinated_moderna,vaccinated_janssen,vaccinated_pfizer,total_vaccinated
0,Malaysia,MYS,<NA>,<NA>,15153452,27880068
1,United States of America,USA,63469471,16886719,112344792,263883327


5. Jakie jest znaczenie wartośći liczbowych z kolumn dotyczących restrykcji (school_closing, workplace_closing, cancel_public_events, restrictions on gatherings)

In [17]:
query = ("""
SELECT DISTINCT date, country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, school_closing, workplace_closing, cancel_public_events, restrictions_on_gatherings
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
ORDER BY country_name
LIMIT 6
""")

df_temp= client.query(query).to_dataframe()
df_temp

,date,Country_name,3_letter_code,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings
0,2020-06-01,Afghanistan,AFG,3,3,2,4
1,2020-03-11,Afghanistan,AFG,0,0,2,0
2,2022-06-23,Afghanistan,AFG,0,2,1,0
3,2022-04-28,Afghanistan,AFG,0,2,1,0
4,2022-08-23,Afghanistan,AFG,<NA>,<NA>,<NA>,<NA>
5,2022-01-14,Afghanistan,AFG,0,2,1,0


<b>School closing:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane zamknięcie lub wprowadzenie obostrzeń,
<br> 2 - nakaz zamknięcia niektórych typów szkół,
<br> 3 - nakaz zamknięcia wszystkich rodzai szkół. <br> <br>
<b>Workplace closing:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane zamknięcie lub praca zdalna, albo wprowadzenie obostrzeń dla firm,
<br> 2 - nakaz zamknięcia niektórych sektorów lub obowiązkowa praca zdalna dla części pracowników,
<br> 3 - nakaz zamknięcia wszystkich miejsc pracy z wyjątkiem działalności niezbędnych (np. sklepy spożywcze. placówki medyczne). <br> <br>
<b>Cancel public events:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane unikanie/odwoływanie,
<br> 2 - zakaz organizacji. <br> <br>
<b>Restrictions on gatherings:</b>
<br> 0 - brak ograniczeń,
<br> 1 - ograniczenia dla bardzo dużych zgromadzeń (powyżej 1000 osób),
<br> 2 - ograniczenia dla zgromadzeń od 101 do 1000 osób,
<br> 3 - ograniczenia dla zgromadzeń od 11 do 100 osób,
<br> 4 - ograniczenia dla zgromadzeń do 10 osób lub mniej.
<br>
via https://github.com/OxCGRT/covid-policy-dataset/blob/main/documentation_and_codebook.md <br>

# Część 4
Przeanalizuj poniższe przypadki. Zastanów się, jakie dane potrzebujesz do każdego z nich, a następnie zapisz je w osobnych, jak najprostszych obiektach DataFrame. Na tym etapie nie przetwarzaj ich. Zadbaj o czystość danych, m.in. usuń niepotrzebne puste wartości i duplikaty, zunifikuj sposób podawania informacji, zaproponuj sposób naprawy błędnych danych. Gotowe dane z obiektów DataFrame zapisz w osobnych plikach CSV.


In [18]:
# 4.1 Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

query = ("""
SELECT DISTINCT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, population, area_sq_km, population_density, population_urban, gdp_per_capita_usd
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0
GROUP BY country_name, iso_3166_1_alpha_3, population, area_sq_km, population_density, population_urban, gdp_per_capita_usd
""")

country_data = client.query(query).to_dataframe()
country_data.to_csv("wyniki\\country_data.csv", index=False)

In [19]:
country_data

,Country_name,3_letter_code,population,area_sq_km,population_density,population_urban,gdp_per_capita_usd
0,Afghanistan,AFG,38928341,652860,59.627,9797273,502
1,Antigua and Barbuda,ATG,97928,440,222.564,23800,17790
2,Angola,AGO,32866267,1246700,26.363,21061025,2973
3,Caribbean Netherlands,BES,26221,322,79.942,<NA>,<NA>
4,Belize,BLZ,397621,22970,17.432,179039,4815
...,...,...,...,...,...,...,...
241,Swaziland,SWZ,1160164,17360,67.451,275333,3837
242,Tokelau,TKL,1350,10,135.000,<NA>,<NA>
243,Trinidad and Tobago,TTO,1399491,5130,272.805,741944,17276
244,British Virgin Islands,VGB,30237,150,201.580,14449,<NA>


In [20]:
# 4.2 Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.

# Tutaj są wartości null w sumie. Można podmienić na 0 COALESCE(SUM(new_confirmed), 0) ale oznaczała by to, że nikt tam nie chorował a naprawdę nie ma danych.
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`, new_confirmed as `Total_confirmed_cases`,
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_confirmed
ORDER BY date
""")

covid_statistic = client.query(query).to_dataframe()
covid_statistic.dropna(subset=['Total_confirmed_cases'], inplace=True)
covid_statistic.to_csv("wyniki\\covid_statistic.csv", index=False)


In [21]:
covid_statistic

,date,3_letter_code,Total_confirmed_cases
0,2020-01-01,LCA,0
1,2020-01-01,FRA,0
2,2020-01-01,VAT,0
3,2020-01-01,MRT,0
4,2020-01-01,SUR,0
...,...,...,...
243193,2022-09-15,IDN,2651
243246,2022-09-15,AUS,1796
243276,2022-09-15,THA,1125
243277,2022-09-15,FRA,33263


In [22]:
# 4.3 Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.

query = ("""
SELECT date,iso_3166_1_alpha_3 AS `3_letter_code`, new_deceased as `Total_new_deceased` 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_deceased
""")

covid_deceased = client.query(query).to_dataframe()
covid_deceased.dropna(subset=['Total_new_deceased'], inplace=True)
covid_deceased.to_csv("wyniki\\covid_deceased.csv", index=False)


In [23]:
covid_deceased

,date,3_letter_code,Total_new_deceased
0,2020-05-04,AFG,23
1,2021-09-12,BIH,32
2,2020-09-19,BGD,40
3,2021-03-28,BOL,21
4,2021-06-19,CHL,133
...,...,...,...
243781,2021-02-08,MOZ,15
243782,2021-04-25,CRI,15
243783,2021-04-24,GTM,15
243784,2021-12-28,MKD,15


In [24]:
# 4.4 Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19. 

query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`,
new_persons_vaccinated as `Total_new_vacinations`,
new_persons_fully_vaccinated as`New_persons_fully_vaccinated`
/*
SUM(new_persons_fully_vaccinated_pfizer) as `Total_fully_vacinations_by_pfizer`,
SUM(new_persons_fully_vaccinated_moderna) as `Total_fully_vacinations_by_moderna`,
SUM(new_persons_fully_vaccinated_janssen) as `Total_fully_vacinations_by_janssen`
*/
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_persons_vaccinated, new_persons_fully_vaccinated
""")

covid_trend_analysis = client.query(query).to_dataframe()
covid_trend_analysis.dropna(subset=['Total_new_vacinations', 'New_persons_fully_vaccinated'], how='all', inplace=True)
covid_trend_analysis.to_csv("wyniki\\covid_trend_analysis.csv", index=False)

In [25]:
covid_trend_analysis

,date,3_letter_code,Total_new_vacinations,New_persons_fully_vaccinated
0,2022-03-31,BEL,181,552
1,2021-06-23,BGR,6138,7761
2,2022-02-24,BOL,12239,16189
3,2021-03-28,BOL,367,403
4,2022-05-21,BOL,1220,1593
...,...,...,...,...
243781,2021-04-30,BOL,0,0
243782,2022-02-04,CHN,0,0
243783,2021-07-13,MHL,0,0
243784,2021-07-27,PLW,263,0


In [26]:
# 4.5 Zdefiniuj własny dodatkowy przypadek.

# Chcemy posidać informacje o liczbe wykonanych testów
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`, CAST(new_tested AS INT64) as `Total_tested`
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, investment_in_vaccines, new_tested
ORDER BY date
""")

investment_in_vaccines = client.query(query).to_dataframe()
investment_in_vaccines.dropna(subset=['Total_tested'], how='all', inplace=True)
investment_in_vaccines.to_csv("wyniki\\investment_in_vaccines.csv", index=False)

display(investment_in_vaccines)

,date,3_letter_code,Total_tested
20,2020-01-01,MEX,72
93,2020-01-01,ARG,4
136,2020-01-01,BRA,1154
373,2020-01-02,ARG,42
375,2020-01-02,BRA,2337
...,...,...,...
243059,2022-09-15,AUT,67392
243204,2022-09-15,MYS,0
243213,2022-09-15,ITA,146983
243255,2022-09-15,AUS,4367


# Część 5

Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [27]:
merge = pd.merge(country_data, covid_statistic, on=['3_letter_code'])
merge = pd.merge(merge, covid_statistic, on=['date', '3_letter_code'])
merge = pd.merge(merge, covid_trend_analysis, on=['date', '3_letter_code'])
merge = pd.merge(merge, investment_in_vaccines, on=['date', '3_letter_code'])
merge.to_csv("wyniki\\merge.csv", index=False)
merge

,Country_name,3_letter_code,population,area_sq_km,population_density,population_urban,gdp_per_capita_usd,date,Total_confirmed_cases_x,Total_confirmed_cases_y,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested
0,Belize,BLZ,397621,22970,17.432,179039,4815,2021-06-23,24,24,3845,4761,931
1,Belize,BLZ,397621,22970,17.432,179039,4815,2021-07-06,18,18,2794,1175,1351
2,Cuba,CUB,11326616,109880,106.413,8739135,8821,2021-05-16,1057,1057,385908,<NA>,21995
3,Cuba,CUB,11326616,109880,106.413,8739135,8821,2021-05-17,1244,1244,84094,<NA>,22648
4,Cuba,CUB,11326616,109880,106.413,8739135,8821,2021-05-18,1339,1339,72374,<NA>,21574
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31513,Trinidad and Tobago,TTO,1399491,5130,272.805,741944,17276,2022-06-12,217,217,98,49,531
31514,Trinidad and Tobago,TTO,1399491,5130,272.805,741944,17276,2022-06-14,121,121,48,35,1809
31515,Trinidad and Tobago,TTO,1399491,5130,272.805,741944,17276,2022-06-18,106,106,85,43,446
31516,Trinidad and Tobago,TTO,1399491,5130,272.805,741944,17276,2022-06-20,101,101,70,70,512


# Część 6


Usunięcie powtarzających się kolumn, usuwamy z naszego zbioru, ponieważ zbiory znalezione potem okazują się być dokładniejsze.

In [28]:
merge.drop('population', axis=1, inplace=True)
merge.drop('area_sq_km', axis=1, inplace=True)
merge.drop('population_density', axis=1, inplace=True)
merge.drop('gdp_per_capita_usd', axis=1, inplace=True)

merge

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases_x,Total_confirmed_cases_y,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested
0,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931
1,Belize,BLZ,179039,2021-07-06,18,18,2794,1175,1351
2,Cuba,CUB,8739135,2021-05-16,1057,1057,385908,<NA>,21995
3,Cuba,CUB,8739135,2021-05-17,1244,1244,84094,<NA>,22648
4,Cuba,CUB,8739135,2021-05-18,1339,1339,72374,<NA>,21574
...,...,...,...,...,...,...,...,...,...
31513,Trinidad and Tobago,TTO,741944,2022-06-12,217,217,98,49,531
31514,Trinidad and Tobago,TTO,741944,2022-06-14,121,121,48,35,1809
31515,Trinidad and Tobago,TTO,741944,2022-06-18,106,106,85,43,446
31516,Trinidad and Tobago,TTO,741944,2022-06-20,101,101,70,70,512


In [29]:
# 6.1 world_countries.csv 

# Tutaj możemy usunąć dane z 4.1, ponieważ są one już w pliku .csv z 4.1 

world_countries = pd.read_csv("world_countries.csv")

merge = pd.merge(merge, world_countries, left_on=['3_letter_code'], right_on=['CCA3'], how="left")
merge.drop('Country/Territory', axis=1, inplace=True)
merge.drop('CCA3', axis=1, inplace=True)

merge

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases_x,Total_confirmed_cases_y,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,...,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
0,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,359871,322106,240406,182589,145133,120905,22966,17.6466,1.0131,0.01
1,Belize,BLZ,179039,2021-07-06,18,18,2794,1175,1351,177,...,359871,322106,240406,182589,145133,120905,22966,17.6466,1.0131,0.01
2,Cuba,CUB,8739135,2021-05-16,1057,1057,385908,<NA>,21995,85,...,11339894,11290417,11105791,10626680,9809107,8869636,109884,102.0366,0.9961,0.14
3,Cuba,CUB,8739135,2021-05-17,1244,1244,84094,<NA>,22648,85,...,11339894,11290417,11105791,10626680,9809107,8869636,109884,102.0366,0.9961,0.14
4,Cuba,CUB,8739135,2021-05-18,1339,1339,72374,<NA>,21574,85,...,11339894,11290417,11105791,10626680,9809107,8869636,109884,102.0366,0.9961,0.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31513,Trinidad and Tobago,TTO,741944,2022-06-12,217,217,98,49,531,153,...,1460177,1410296,1332203,1266518,1127852,988890,5130,298.4491,1.0035,0.02
31514,Trinidad and Tobago,TTO,741944,2022-06-14,121,121,48,35,1809,153,...,1460177,1410296,1332203,1266518,1127852,988890,5130,298.4491,1.0035,0.02
31515,Trinidad and Tobago,TTO,741944,2022-06-18,106,106,85,43,446,153,...,1460177,1410296,1332203,1266518,1127852,988890,5130,298.4491,1.0035,0.02
31516,Trinidad and Tobago,TTO,741944,2022-06-20,101,101,70,70,512,153,...,1460177,1410296,1332203,1266518,1127852,988890,5130,298.4491,1.0035,0.02


In [30]:
gdp = pd.read_csv("gdp.csv")

merge = pd.merge(merge, gdp, left_on=['3_letter_code'], right_on=['Country Code'], how="left")
merge.drop('Country Name', axis=1, inplace=True)
merge.drop('Country Code', axis=1, inplace=True)

merge['Year'] = merge['Year'].astype('Int64') 

merge.to_csv("wyniki\\final.csv", index=False)

merge

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases_x,Total_confirmed_cases_y,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,...,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,Year,Value
0,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,240406,182589,145133,120905,22966,17.6466,1.0131,0.01,1960,2.807189e+07
1,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,240406,182589,145133,120905,22966,17.6466,1.0131,0.01,1961,2.996437e+07
2,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,240406,182589,145133,120905,22966,17.6466,1.0131,0.01,1962,3.185692e+07
3,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,240406,182589,145133,120905,22966,17.6466,1.0131,0.01,1963,3.374941e+07
4,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,240406,182589,145133,120905,22966,17.6466,1.0131,0.01,1964,3.619383e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500224,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,1332203,1266518,1127852,988890,5130,298.4491,1.0035,0.02,2012,2.569416e+10
1500225,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,1332203,1266518,1127852,988890,5130,298.4491,1.0035,0.02,2013,2.643616e+10
1500226,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,1332203,1266518,1127852,988890,5130,298.4491,1.0035,0.02,2014,2.617595e+10
1500227,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,1332203,1266518,1127852,988890,5130,298.4491,1.0035,0.02,2015,2.355922e+10


# Część 7

### Dane związane z inflacją

In [31]:
# https://www.worldbank.org/en/research/brief/inflation-database

# Z wszystkich strony arkusza wybraliśmy ogólny przyrost inflacji w danych latach
inflation_csv = pd.read_excel("load_data\\inflation_all.xlsx", sheet_name='hcpi_a')

inflation_csv.drop(['IMF Country Code', 'Indicator Type', 'Series Name', 'Note', 'Country'], axis=1, inplace=True)

inflation_transposed = inflation_csv.melt(id_vars=[inflation_csv.columns[0]], 
                                          var_name="Year", 
                                          value_name="Inflation")

inflation_transposed.rename(columns={inflation_csv.columns[0]: "Country Code"}, inplace=True)

inflation_transposed.dropna(subset=['Inflation'], inplace=True)
inflation_transposed


,Country Code,Year,Inflation
0,AFG,1970,25.510000
2,DZA,1970,6.600000
3,AGO,1970,7.970000
4,ATG,1970,8.870000
5,ARG,1970,13.586900
...,...,...,...
11063,VNM,2023,3.252893
11064,PSE,2023,5.871102
11065,YEM,2023,-1.225000
11066,ZMB,2023,10.953000


In [32]:
merge_inflation = pd.merge(merge, inflation_transposed, left_on=['3_letter_code', 'Year'], right_on=['Country Code', 'Year'], how='left',)
merge_inflation.drop('Country Code', axis=1, inplace=True)

merge_inflation.to_csv("wyniki\\merge_inflation.csv", index=False)
merge_inflation

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases_x,Total_confirmed_cases_y,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,...,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,Year,Value,Inflation
0,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,182589,145133,120905,22966,17.6466,1.0131,0.01,1960,2.807189e+07,NaN
1,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,182589,145133,120905,22966,17.6466,1.0131,0.01,1961,2.996437e+07,NaN
2,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,182589,145133,120905,22966,17.6466,1.0131,0.01,1962,3.185692e+07,NaN
3,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,182589,145133,120905,22966,17.6466,1.0131,0.01,1963,3.374941e+07,NaN
4,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,182589,145133,120905,22966,17.6466,1.0131,0.01,1964,3.619383e+07,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500224,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,1266518,1127852,988890,5130,298.4491,1.0035,0.02,2012,2.569416e+10,9.288705
1500225,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,1266518,1127852,988890,5130,298.4491,1.0035,0.02,2013,2.643616e+10,5.149316
1500226,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,1266518,1127852,988890,5130,298.4491,1.0035,0.02,2014,2.617595e+10,5.689053
1500227,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,1266518,1127852,988890,5130,298.4491,1.0035,0.02,2015,2.355922e+10,4.659380


### Dane o bezrobociu

In [33]:
#  https://data.worldbank.org/indicator/SL.UEM.TOTL.ZS?view=map&year=1991

total_unemployment = pd.read_csv("load_data\\total_unemployment.csv")
total_unemployment.drop(['Country Name', 'Indicator Name', 'Indicator Code', 'Unnamed: 68'], axis=1, inplace=True)

total_unemployment = total_unemployment.melt(id_vars=['Country Code'], var_name="Year", value_name="Unemployment")

total_unemployment.dropna(subset=['Unemployment'], inplace=True)

total_unemployment


,Country Code,Year,Unemployment
8247,AFE,1991,8.179629
8248,AFG,1991,8.070000
8249,AFW,1991,4.158680
8250,AGO,1991,16.855000
8251,ALB,1991,10.304000
...,...,...,...
17018,WSM,2023,4.977000
17020,YEM,2023,17.091000
17021,ZAF,2023,32.098000
17022,ZMB,2023,5.905000


In [34]:
merge_unemployment = pd.merge(merge_inflation, total_unemployment, left_on=['3_letter_code', 'Year'], right_on=['Country Code', 'Year'] , how='left',)
merge_unemployment.drop('Country Code', axis=1, inplace=True)

merge_unemployment.to_csv("wyniki\\total_unemployment.csv", index=False)
merge_unemployment

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases_x,Total_confirmed_cases_y,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,...,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,Year,Value,Inflation,Unemployment
0,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,145133,120905,22966,17.6466,1.0131,0.01,1960,2.807189e+07,NaN,NaN
1,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,145133,120905,22966,17.6466,1.0131,0.01,1961,2.996437e+07,NaN,NaN
2,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,145133,120905,22966,17.6466,1.0131,0.01,1962,3.185692e+07,NaN,NaN
3,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,145133,120905,22966,17.6466,1.0131,0.01,1963,3.374941e+07,NaN,NaN
4,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,145133,120905,22966,17.6466,1.0131,0.01,1964,3.619383e+07,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500224,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,1127852,988890,5130,298.4491,1.0035,0.02,2012,2.569416e+10,9.288705,NaN
1500225,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,1127852,988890,5130,298.4491,1.0035,0.02,2013,2.643616e+10,5.149316,NaN
1500226,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,1127852,988890,5130,298.4491,1.0035,0.02,2014,2.617595e+10,5.689053,NaN
1500227,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,1127852,988890,5130,298.4491,1.0035,0.02,2015,2.355922e+10,4.659380,NaN


### Dane o przestępstwach w latach

In [35]:
crime = pd.read_csv("load_data\\crime_rate_per_100k.csv")

crime_melt = (
    crime
    .drop(['Series Name', 'Series Code', 'Country Name'], axis=1)
    .melt(id_vars=['Country Code'], var_name="Year", value_name="Crime Rate")
    .assign(
        Year=lambda df: df['Year'].str.replace(r'\[.*\]', '', regex=True).astype(int),
        **{"Crime Rate": lambda df: pd.to_numeric(df['Crime Rate'], errors='coerce')} # coerce zmienia na Nan jak są dziwne wartości
    )
    .dropna(subset=['Crime Rate'])
)

crime_melt

,Country Code,Year,Crime Rate
0,IND,1998,4.825762
1,UKR,1998,8.432860
3,ALB,1998,20.419683
10,ARM,1998,2.869883
12,AUS,1998,1.795538
...,...,...,...
13415,SSF,2022,6.198019
13416,SSA,2022,6.198019
13417,TSS,2022,6.198019
13418,UMC,2022,6.058378


In [36]:
merge_crime = pd.merge(merge_unemployment, crime_melt, left_on=['3_letter_code', 'Year'], right_on=['Country Code', 'Year'] , how='left',)
merge_crime.drop('Country Code', axis=1, inplace=True)

merge_crime.to_csv("wyniki\\oddaj.csv", index=False)
merge_crime

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases_x,Total_confirmed_cases_y,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,...,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,Year,Value,Inflation,Unemployment,Crime Rate
0,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,120905,22966,17.6466,1.0131,0.01,1960,2.807189e+07,NaN,NaN,NaN
1,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,120905,22966,17.6466,1.0131,0.01,1961,2.996437e+07,NaN,NaN,NaN
2,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,120905,22966,17.6466,1.0131,0.01,1962,3.185692e+07,NaN,NaN,NaN
3,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,120905,22966,17.6466,1.0131,0.01,1963,3.374941e+07,NaN,NaN,NaN
4,Belize,BLZ,179039,2021-06-23,24,24,3845,4761,931,177,...,120905,22966,17.6466,1.0131,0.01,1964,3.619383e+07,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1971186,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,988890,5130,298.4491,1.0035,0.02,2014,2.617595e+10,5.689053,NaN,2.477000
1971187,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,988890,5130,298.4491,1.0035,0.02,2015,2.355922e+10,4.659380,NaN,28.763646
1971188,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,988890,5130,298.4491,1.0035,0.02,2015,2.355922e+10,4.659380,NaN,2.428000
1971189,Trinidad and Tobago,TTO,741944,2022-06-21,145,145,3,0,1900,153,...,988890,5130,298.4491,1.0035,0.02,2016,2.189471e+10,3.094139,NaN,31.510972
